In [ ]:
import pandas as pd
import json
import os

# Load JSON data
with open("../output/accounts_master.json", "r", encoding="utf-8") as f:
    data = json.load(f)

accounts = data["accounts"]
df = pd.json_normalize(accounts)

# Normalize manager data
def extract_countries(managers):
    if not isinstance(managers, list):
        return []
    return [m["country"] for m in managers]

df["admin_countries"] = df["page_managers"].apply(extract_countries)

# Identify pages with foreign admins
df["has_foreign_admins"] = df["admin_countries"].apply(lambda countries: any(c != "Romania" for c in countries))
df["foreign_only"] = df["admin_countries"].apply(lambda countries: all(c != "Romania" for c in countries) and countries)

# Filter relevant pages
foreign_pages = df[df["has_foreign_admins"]].copy()
foreign_pages["followers"] = pd.to_numeric(foreign_pages["stats.followers"], errors="coerce")

# Markdown-formatted name
foreign_pages["name"] = foreign_pages.apply(
    lambda row: f"[{row['name']}]({row['profile_url']})", axis=1
)

# Add number of foreign countries
foreign_pages["foreign_country_count"] = foreign_pages["admin_countries"].apply(
    lambda countries: len([c for c in countries if c != "Romania"])
)

# Report columns
report_df = foreign_pages[[
    "name", "followers", "admin_countries", "foreign_country_count", "foreign_only"
]].sort_values(by=["foreign_country_count", "followers"], ascending=[False, False])

# Save report
os.makedirs("../reports", exist_ok=True)
with open("../reports/foreign_admin_pages.md", "w", encoding="utf-8") as report:
    report.write("# Pages with Foreign Admins\n\n")
    report.write(
        "This report lists pages that have at least one admin outside of Romania.\n\n"
        "- `foreign_only = True` means **no admin is from Romania**.\n"
        "- Pages are sorted by number of foreign admin countries and follower count.\n\n"
    )
    report.write(report_df.to_markdown(index=False))
